# Tool to track cells manually across sessions

This is a tool that helps tracking cells manually across many sessions. It shows the reference cell on the left side of the graph, and shows all possible matching cells from other sessions on the right side of the graph. The result is an array that contains the IDs of each tracked cell in all tracked sessions.

## Import scripts

First we have to import all the scripts that the tool uses.

In [1]:
import sys
sys.path.append('../custom scripts/')
import numpy as np
import multisession_analysis.multisession_registration as tracker

C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

## Select sessions

Now we have to determine which sessions should be processed. For this, we have to initialize a list (`session_list`) that holds the path to every session folder. Additionally, we have to say which session should be the reference session by giving the date string to the `reference_session` variable. From this session, all place cells will be tracked across all other sessions from `session_list`, and these cells will be the "reference cells" in the interactive plot.

In [2]:
# Which sessions should be aligned?
session_list = [r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 1\Frontal',
                r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 2\Frontal',
                r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 3\Frontal',
                r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 4\Frontal',
                r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 5\Frontal',]


# Which session should be the reference (place cells from this session will be tracked)
reference_session = 'Session 1'

## Load the data

Now that we know which sessions we want to track, we have to load the data from these sessions. This can take some time depending on how many sessions we chose and how many cells are in the field of view.

In [3]:
# This function loads the data of all sessions and stores it in the list "pcf_objects"
spatial, templates, dim, pcf_objects = tracker.load_multisession_data(session_list, place_cell_mode=False)

Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 1\Frontal\cnm_results.hdf5...
Successfully loaded data from Pre_Stroke\Session 1\Frontal (1/5).
Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 2\Frontal\cnm_results.hdf5...
Successfully loaded data from Pre_Stroke\Session 2\Frontal (2/5).
Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 3\Frontal\cnm_results.hdf5...
Successfully loaded data from Pre_Stroke\Session 3\Frontal (3/5).
Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 4\Frontal\cnm_results.hdf5...
Successfully loaded data from Pre_Stroke\Session 4\Frontal (4/5).
Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 5\Frontal\cnm_results.hdf5...
Successfully loaded data from Pre_Stroke\Session 5\Frontal (5/5).


## Prepare data for tracking tool

The data is now loaded, but it still has to be processed a bit to have the right format for the manual tracking tool. You dont have to change anything here, the parameters should not change, just run the code below.

In [4]:
# This function prepares the data for the tracking process
%matplotlib inline
target_session_list, place_cell_indices, alignment_array, all_contours_list, all_shifts_list = tracker.prepare_manual_alignment_data(
                                                                                                          pcf_objects, reference_session,
                                                                                                          place_cell_mode=False,
                                                                                                          session_list=session_list)

%matplotlib qt

In case you already started to track the cells from this session, saved the `alignment_array`, but could not finish, you can load this half-filled `alignment_array` with the code block below. You dont have to run this code if you did not process this session already. To load the correct alignment array, provide the complete path of the file directory, file name and also the extension to the variable `file_path`.

In [5]:
# If you started to align the place cells of this session, but saved the table incomplete and want to continue, you can
# load the table with this function and pick up where you left. Note that the file path should include the 
# file name and extension
file_path = r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 1\Caudal\pc_alignment_Session 1_Session 1.txt'
alignment_array = np.loadtxt(file_path)

## Run the tool

Finally all the data is ready, and we can run the tool. It creates a figure with two sides. On the left side, a big image of the current reference cell is drawn. On the left side, all possible matching cells are drawn. Matching cells are cells that have a center of mass closest to the reference cell. Also, the cells are ordered by distance, meaning that the top left candidate cell is the closest, and the bottom right cell is the farthest cell from the reference neuron. Only the contours of the reference and candidate cells are drawn, but you can still move the FOVs around to find possible landmarks.

Click the candidate cell that you think is the same neuron as the reference. Then the next cell is loaded automatically and your choice is saved in the `alignment_array`, which is automatically updated. This is also printed out, with the formula `Reference cell (index of reference cell in this session), clicked neuron (idx reference, idx session, idx matched neuron)`. If you cannot find a reference cell in a certain session (because the cell was not active or the FOV changed too much), you can click on a plot saying "No Match", and the cell ID is saved as `-10`, the code for "not found".

The `alignment_array` is updated automatically after every click. This means that if you close the interactive plot by accident, you can just run the code below again and the tool will pick up where you left.
Similarly, if you did a mistake while tracking (e.g. misclicked the wrong cell), you can manually remove the entry. To do this, look at the last printed line (or the line where you made the mistake) and use the two first numbers in the brackets to index the `alignment_array`. For example, if you made a mistake in your previous click and the last line says `clicked neuron (0, 7, 150)`, you can remove this choice by running the command `alignment_array[0, 7] = -1` in the second code box below. Then you close the interactive plot and run the tool again, and it will allow you to choose another cell.

In [7]:
%matplotlib qt

In [8]:
# This is the main function for the tracking. It creates the interactive plot and saves the results in the
# alignment_array, which has one place cell for each row, one session in each column, and each entry is the 
# neuron ID that the reference cell has in each session.
alignment_array = tracker.manual_place_cell_alignment(pcf_sessions=pcf_objects,
                                                      target_sessions=target_session_list,
                                                      cell_idx=place_cell_indices,
                                                      alignment=alignment_array,
                                                      all_contours=all_contours_list,
                                                      all_shifts=all_shifts_list,
                                                      ref_sess=reference_session,
                                                      dim=dim,
                                                      place_cell_mode=False,
                                                      show_neuron_id=True)

Reference cell 10, clicked neuron (10, 1, 10).
Reference cell 10, clicked neuron (10, 2, 29).
Reference cell 10, clicked neuron (10, 3, 9).
Reference cell 10, clicked neuron (10, 4, 7).


In [ ]:
# This code removes a matched neuron ID. Run this code if you made a mistake and want to choose another 
# matching cell. Use the two first numbers in the last printed line to index alignment_array
alignment_array[7, 1] = -1

Sometimes the correct cell does not show up as an option, although you can see it, or sometimes the field-of-view is shifted too much and the tool gives you wrong candidates. In this case, you can run the code below to plot all possible cells in the target session. Then you can manually look for the correct cell. If you found it, use the number displayed in or next to the contour and manually update the `alignment_array`. To do this, use the code line above, just this time use the index of the reference cell (in the title of the left plot in the interactive tool) and the index of the target session (in the title of the right plot). After the `=`, you do not put `-1`, but instead put the neuron ID of the matched cell. 

__IMPORTANT!__ The cell IDs on the right plot of this figure are numbered differently. Subtract 1 from it before updating it into the `alignment_array`. Example: You have found that for reference cell 1, the matching cell from target session 5 has the displayed number 170. Accordingly, you update the `alignment_array` like this: `alignment_array[1, 5] = 169`.

In [ ]:
# In case the correct cell is not displayed in the alignment plot, you can show the whole FOVs of the reference
# as well as the target session and look for the correct cell yourself. Use the indices displayed in the 
# interactive graph to access the correct session from the list and the correct cell ID.
tracker.show_whole_fov(reference_session=pcf_objects[0], target_session=pcf_objects[2], ref_cell_id=9,
                       place_cell_mode=False)


When you are done, you can save your `alignment_array` with the code below. Define the folder in which you want to save the file. The array is saved as a .txt file.

In [9]:
# Save the alignment array under the provided directory as a csv table. Every row is one place cell from the 
# reference session, every column is one session, and the entries are the IDs of each cell in the corresponding
# session.
file_directory = r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch 3\M31\Pre_Stroke\Session 1\Caudal'
tracker.save_alignment(file_directory, alignment_array, reference_session, pcf_objects, place_cell_mode=False)


File [...]dal\pc_alignment_Session 1_Session 1.txt already exists!
Overwrite? [y/n] n
Saving cancelled.


## Plotting traces of aligned cells

A basic way to show aligned cells is to plot the traces of individual cells across sessions. For this you have to load the alignment files that you created before. Then you can filter these cells, e.g. for cells that were recognized in a certain number of sessions. In the end the traces are plotted across sessions, with potential place fields marked in red.